# Description

Link [here](https://www.kaggle.com/c/nlp-getting-started/overview)

Pretrined Bert + untrained GRU + nn.Linear

Train: 1 epoch

In [11]:
! pip install transformers

     |████████████████████████████████| 1.9MB 5.2MB/s 
     |████████████████████████████████| 890kB 21.8MB/s 
     |████████████████████████████████| 3.2MB 35.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=8c1f3c2af014ee39e59e10f4073c72427f7233a2cf8a4f56b59caf5da12c49a5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
tokens = tokenizer.tokenize('लॉन्च हो चुका है. इस स्मार्टफ़ोन में 5G कनेक्टिविटी के लिए MediaTek Dimnesty 800 U चिपसेट दिया गया है. इस वेरिएंट में कंपनी ने 120Hz रिफ़्रेश रेट वाली डिस्प्ले दी है.')

print(tokens)

['[UNK]', 'ह', '##ो', 'च', '##क', '##ा', 'ह', '.', '[UNK]', '[UNK]', 'म', '5', '##g', 'क', '##न', '##क', '##ट', '##ि', '##व', '##ि', '##ट', '##ी', 'क', 'ल', '##ि', '##ए', 'media', '##tek', 'dim', '##nes', '##ty', '800', 'u', 'च', '##ि', '##प', '##स', '##ट', 'द', '##ि', '##य', '##ा', 'ग', '##य', '##ा', 'ह', '.', '[UNK]', 'व', '##र', '##ि', '##ए', '##ट', 'म', 'क', '##प', '##न', '##ी', 'न', '120', '##h', '##z', '[UNK]', 'र', '##ट', 'व', '##ा', '##ल', '##ी', 'ड', '##ि', '##स', '##प', '##ल', 'द', '##ी', 'ह', '.']


### Add stopwords


In [ ]:
DIR = '/content/drive/Shareddrives/Inter-IIt-Tech-meet-9-NLP/workingDir/dataset/'

for file in ['hinglishStopWords.txt', 'hindiStopWords.txt']:
    file = DIR + file
    with open(file, 'r') as f:
        words = f.read().split('\n')
        # print(words[:3])
    tokenizer.add_tokens(words)

# tokenizer.add_tokens(['लॉन्च'])
# tokenizer.add_tokens(['आप'])

In [93]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        self.bert.resize_token_embeddings(len(tokenizer)) 
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [94]:
HIDDEN_DIM = 256
OUTPUT_DIM = 2
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [95]:
device = 'cuda'

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [17]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

### Text dataset

In [3]:
WORKING_DIR = '/content/drive/Shareddrives/Inter-IIt-Tech-meet-9-NLP/workingDir/'

import os
os.chdir(WORKING_DIR)


import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import torch 
import torch.nn as nn
import torchtext
import matplotlib.pyplot as plt


In [6]:
df = pd.read_csv(WORKING_DIR + 'dataset/Cleaned_tweets.csv')[['Tweet_ID', 'Mobile_Tech_Tag', 'Cleaned_tweets']]
df.tail()

,Tweet_ID,Mobile_Tech_Tag,Cleaned_tweets
2187,tweet_3992,1,ने 5g सेगमेंट में एक धांसू फोन लॉन्च किया ...
2188,tweet_3993,1,"भारत में लॉन्च हुआ xiaomi mi 10i, यह है 2021 क..."
2189,tweet_3998,1,चीनी मोबाइल कंपनी रियलमी ने अपना realme v15 5g...
2190,tweet_3999,1,aapne phone hi galat choose kr liya hai..sor...
2191,tweet_4000,1,पर bhaib5g इंडिया में कब आयेगा। ; dosto 2021 m...


In [7]:
df.groupby('Mobile_Tech_Tag').count()

,Tweet_ID,Cleaned_tweets
Mobile_Tech_Tag,,
0,1728,1724
1,464,464


In [8]:
from torchtext.legacy import data

import re
import string 

def clean_text(text):
    text = text.lower()
    #foction de replacement
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub(r"[-()\"#/@;:<>{}=~|.?!,]", "", text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [9]:
from torch.utils.data import Dataset, DataLoader

In [19]:
class TextDataset(Dataset):
    def __init__(self, df, tok):
        self.df = df
        self.tok = tok
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, id):
        text = self.df.iloc[id]['Cleaned_tweets']
        text = self.tok.encode(text, return_tensors='pt')
        label = torch.tensor([self.df.iloc[id].get('Mobile_Tech_Tag', 0)])
        return text, label, self.df.iloc[id]['Tweet_ID']


In [26]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=0, stratify=df[['Mobile_Tech_Tag']])


In [27]:
print('Train: ', train_df.groupby('Mobile_Tech_Tag').count(), '\n\n', 'Val: ', val_df.groupby('Mobile_Tech_Tag').count())

Train:                   Tweet_ID  Cleaned_tweets
Mobile_Tech_Tag                          
0                    1382            1379
1                     371             371 

 Val:                   Tweet_ID  Cleaned_tweets
Mobile_Tech_Tag                          
0                     346             345
1                      93              93


In [58]:
print('Train: ', len(train_df), ' Val: ', len(val_df))

Train:  1753  Val:  439


In [55]:
train_df[train_df.isna().any(axis=1)]

,Tweet_ID,Mobile_Tech_Tag,Cleaned_tweets
471,tweet_0662,0,NaN
866,tweet_1328,0,NaN
107,tweet_0131,0,NaN


In [69]:
train_df.dropna(inplace=True)

In [57]:
val_df[val_df.isna().any(axis=1)]


,Tweet_ID,Mobile_Tech_Tag,Cleaned_tweets
663,tweet_0978,0,NaN


In [70]:
val_df.dropna(inplace=True)

In [71]:
train_dataset = TextDataset(train_df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=1,
                        shuffle=True, num_workers=0)

val_dataset = TextDataset(val_df, tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=1,
                        shuffle=True, num_workers=0)

In [72]:
for batch in train_dataloader:
    print(batch[0].shape, batch[1].shape)
    print(batch[0])
    # [batch_size, max_length_of_sentence_in_batch]
    break

torch.Size([1, 1, 46]) torch.Size([1, 1])
tensor([[[  101,  1037,  2047,  2522, 27794,  1997,  7027,  9387,  2003,  9343,
           1998,  4855,  1041, 15549,  7368,  2157,  2013,  2037, 26381,  2015,
           1517, 10523,  1996,  6202,  2004,  1037,  4066,  1997,  2678,  2208,
           1010,  2714,  2000, 11777,  1037,  2047,  2152,  3556,  2006,  1037,
          11989, 10439,  1010,  1998,  4339,   102]]])


In [73]:
for batch in train_dataset:
    print(batch, batch[0].shape, batch[1].shape, sep='\n')
    break

(tensor([[  101,  3437,  1024,  1011,  2057,  2024,  1996, 13200,  1997,  8412,
          1006, 10047,  2595,  2575, 20842,  1007,  4185,  8737, 26381,  4950,
          2974,  1999,  2634,  1012,  5221,  4299,  2000,  2663,   100,   102]]), tensor([1]), 'tweet_3455')
torch.Size([1, 30])
torch.Size([1])


In [90]:
from tqdm.autonotebook import tqdm

def train(model, iterator, optimizer, criterion, phase='train'):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator, leave=False, position=0):
        
        optimizer.zero_grad()

        # print(batch)
        text_ids = batch[0].to(device)
        label = batch[1].to(device)
        
        predictions = model(text_ids)

        # print(f'Input: {text_ids.shape}, Labels: {label.shape}, Outs: {predictions.shape}\n')
        # print(text_ids, label, predictions)
        
        loss = criterion(predictions, label)
        # acc = binary_accuracy(predictions, label)
        acc = torch.sum(torch.argmax(predictions, 1) == label.data)
        
        if phase == 'train':
            loss.backward()
            optimizer.step()
        
        epoch_loss += loss.item() * label.shape[0]
        epoch_acc += acc.item() 
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [96]:
for epoch in range(3):
    loss, acc = train(model, train_dataset, optimizer, criterion=criterion)
    print(f'\nEpoch: {epoch}, Acc: {acc:.2f}, Loss: {loss:.2f}\n')


Epoch: 0, Acc: 0.93, Loss: 0.22




Epoch: 1, Acc: 0.96, Loss: 0.10




Epoch: 2, Acc: 0.97, Loss: 0.08



In [97]:
for epoch in range(3):
    loss, acc = train(model, val_dataset, optimizer, criterion, phase='val')
    print(f'\nEpoch: {epoch}, Acc: {acc:.2f}, Loss: {loss:.2f}\n')


Epoch: 0, Acc: 0.97, Loss: 0.06




Epoch: 1, Acc: 0.97, Loss: 0.08




Epoch: 2, Acc: 0.97, Loss: 0.09



In [85]:
for batch in train_dataloader:
    print(batch[0].shape, batch[1].shape)
    break

torch.Size([1, 1, 49]) torch.Size([1, 1])


#### Note:
- Each class didn't had same number of data points
- Only in 3 epochs, accuracy is decent (given the fact that GRU and nn.Linear was randomly initialized)
- Not sure if total number of data points is good enough in number
- Not sure how good have the embeddings for Hindi/ Hinglish been trained (as the number of data points corresponding to those languages are comparatively less). Luckily found some set of StopWords for these languages, which hopefully makes the bert tokenization task better.

### Predict

In [ ]:
test_df = pd.read_csv('cleaned test.csv')
test_df.tail()

,id,keyword,location,text
3258,10861,NaN,NaN,earthquake safety los angeles ûò safety faste...
3259,10865,NaN,NaN,storm in ri worse than last hurricane my hard...
3260,10868,NaN,NaN,green line derailment in chicago
3261,10874,NaN,NaN,meg issues hazardous weather outlook hwo
3262,10875,NaN,NaN,cityofcalgary has activated its municipal emer...


In [ ]:
test_dataset = TextDataset(test_df, tokenizer)

In [ ]:
for batch in test_dataset:
    print(batch, '|', batch[-1].item(), '\n')
    out = model(batch[0].cuda())

    print(torch.argmax(out, dim=1).item(), out, sep='\t')
    break

(tensor([[ 101, 2074, 3047, 1037, 6659, 2482, 5823,  102]]), tensor([0]), 0) | 0 

1	tensor([[-0.5530,  0.5125]], device='cuda:0', grad_fn=<AddmmBackward>)


In [ ]:
preds = pd.DataFrame(columns=['id', 'target'])

for batch in tqdm(test_dataset):
    out = model(batch[0].cuda())
    out = torch.argmax(out, dim=1).item()

    preds = preds.append({'id': batch[-1].item(), 'target': out}, ignore_index=True)

In [ ]:
preds.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
org = set(pd.read_csv('test.csv')['id'])
org == set(preds['id']), len(org - set(preds['id'])), len(org), len(preds)

(True, 0, 3263, 3263)

In [ ]:
preds.to_csv('preds.csv', index=False)

In [ ]:
# extra

set(preds['id']) - set(test_df['id'])

In [ ]:
len(set(preds['id'])), len(set(test_df['id']))

In [ ]:
sorted(list(preds['id']), reverse=True)[:20], sorted(list(test_df['id']), reverse=True)[:20]

In [ ]:
for x in test_df['id']: 
    if x not in preds['id']:
        print(x)

In [ ]:
{'Rank': 874, 'Score': 0.79129, 'Epoch': 4}

{'Epoch': 4, 'Rank': 874, 'Score': 0.79129}

#### References
- Hinglish/ Hindi Stop Words [github](https://github.com/TrigonaMinima/HinglishNLP/tree/master/data/assets)